In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-219478
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-219478


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

#cluster_name = "GIVE_A_CLUSTER_NAME"
cluster_name = "vag1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    cpu_cluster=ComputeTarget(workspace=ws,name=cluster_name)
    print('Cluster already exists!')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",max_nodes=4)
    cpu_cluster = ComputeTarget(ws,cluster_name,compute_config)
cpu_cluster.wait_for_completion(show_output=True)

Cluster already exists!

Running


In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
parameter_space={
    '--C':choice(0.01,0.1,1.0,10.,100.),
    "--max_iter":choice(100,500,1000)
}
ps = RandomParameterSampling(parameter_space)### YOUR CODE HERE ###

# Specify a Policy
# search more on BanditPolicy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.15, slack_amount=None, delay_evaluation=5)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")
print(parameter_space['--C'][1][0])
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                      # what is those arguments
#                            arguments=['--C', parameter_space['--C'][1][0], '--max_iter', parameter_space['--max_iter'][1][0]],
                            compute_target=cpu_cluster,
                            environment=sklearn_env)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                    hyperparameter_sampling=ps,
                    policy=policy,
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=4,
                    max_concurrent_runs=4)### YOUR CODE HERE ###

[0.01, 0.1, 1.0, 10.0, 100.0]


In [21]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
run_hyperdrive=exp.submit(config=hyperdrive_config)
run_hyperdrive.wait_for_completion(show_output=True)
RunDetails(run_hyperdrive).show()

RunId: HD_d58ad425-7380-4edc-ac99-8117eb9b9ce2
Web View: https://ml.azure.com/runs/HD_d58ad425-7380-4edc-ac99-8117eb9b9ce2?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-219324/workspaces/quick-starts-ws-219324&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-12-19T20:08:13.625529][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-12-19T20:08:14.4909841Z][SCHEDULER][INFO]Scheduling job, id='HD_d58ad425-7380-4edc-ac99-8117eb9b9ce2_0' 
[2022-12-19T20:08:14.5889766Z][SCHEDULER][INFO]Scheduling job, id='HD_d58ad425-7380-4edc-ac99-8117eb9b9ce2_1' 
[2022-12-19T20:08:14.7314573Z][SCHEDULER][INFO]Scheduling job, id='HD_d58ad425-7380-4edc-ac99-8117eb9b9ce2_2' 
[2022-12-19T20:08:14.8260710Z][SCHEDULER][INFO]Scheduling job, id='HD_d58ad425-7380-4edc-ac99-8117eb9b9ce2_3' 
[2022-12-19T20:08:14.760066][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [35]:
import joblib
# Get your best run and save the model from that run.
best_run = run_hyperdrive.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Best run metrics:', best_run_metrics)
print('\n parameter values:', parameter_values)
#print('\n learning rate:',parameter_values[3])
#print('\n keep probability:',parameter_values[5])
#print('\n batch size:',parameter_values[7])
### YOUR CODE HERE ###

os.makedirs('outputs', exist_ok=True)

#joblib.dump(run_hyperdrive, filename='./outputs/best_model.pkl')

saved_model=best_run.register_model(model_name='best-hyperdrive',model_path='./')

Best Run Id:  HD_d58ad425-7380-4edc-ac99-8117eb9b9ce2_0

 Best run metrics: {'Regularization Strength:': 0.1, 'Max iterations:': 100, 'Accuracy': 0.9111785533636824}

 parameter values: ['--C', '0.1', '--max_iter', '100']


In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
data_bank=TabularDatasetFactory.from_delimited_files(path)
### YOUR CODE HERE ###

In [8]:
from train import clean_data
from azureml.core import Dataset
# Use the clean_data function to clean your data.
x, y = clean_data(data_bank)### YOUR DATA OBJECT HERE ###

# split into train and test data
from sklearn.model_selection import train_test_split
import pandas as pd
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=42)

#data_train=pd.concat([x_train,y_train],axis=1)
#data_test=pd.concat([x_test,y_test],axis=1)
data_train=x_train.copy()
data_test=x_test.copy()
data_train['y']=y_train
data_test['y']=y_test
ds=data_train.to_csv('training/train.csv')
ds=ws.get_default_datastore()
ds.upload(src_dir='training',target_path='training',overwrite=True)
ds_train=Dataset.Tabular.from_delimited_files(path=[(ds,'training/train.csv')])

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


In [9]:
ds_train

{
  "source": [
    "('workspaceblobstore', 'training/train.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    compute_target=cpu_cluster,
    training_data=ds_train,
    label_column_name='y',
    n_cross_validations=5,
    enable_early_stopping=True)

In [11]:
# Submit your automl run
run=exp.submit(automl_config, show_output=True)
run.wait_for_completion(show_output=True)
### YOUR CODE HERE ###

Submitting remote run.
No run_configuration provided, running on vag1 with default configuration
Running on remote compute: vag1


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_088e2b1a-fadc-4fd9-85bd-5bbc40ae14df,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_088e2b1a-fadc-4fd9-85bd-5bbc40ae14df,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2582                          |1                               |23065                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_088e2b1a-fadc-4fd9-85bd-5bbc40ae14df',
 'target': 'vag1',
 'status': 'Completed',
 'startTimeUtc': '2022-12-20T15:44:55.54657Z',
 'endTimeUtc': '2022-12-20T16:27:55.517833Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'vag1',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-219478","workspace_name":"quick-starts-ws-219478","region":"southcentralus","compute_target":"vag1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true

In [12]:
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [13]:
# Retrieve and save your best automl model.
best_model,fit_model = run.get_output()
print('best model:',best_model)
print('fit model:',fit_model)

### YOUR CODE HERE ###
# saving the model
#os.makedirs('outputs', exist_ok=True)

#joblib.dump(value=best_run, filename='outputs/best_model_automl.pkl')

saved_model=best_model.register_model(model_name='bestmodel_automl',model_path='')

best model: Run(Experiment: udacity-project,
Id: AutoML_088e2b1a-fadc-4fd9-85bd-5bbc40ae14df_30,
Type: azureml.scriptrun,
Status: Completed)
fit model: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(cop

In [15]:
print('model name',best_model.properties['model_name'])
print('accuracy',best_model.get_metrics()['accuracy'])

model name AutoML088e2b1af30
accuracy 0.9174073271190114


In [ ]:
# test the best model
#from sklearn.metrics import accuracy_score
#y_pred=best_model.predict(x_test)
#acc=accuracy_score(y_pred,y_test)

In [ ]:
# deleting the compute cluster

cpu_cluster.delete()